# Динамическая выпуклая оболочка

> **Выпуклая оболочка** мн-ва точек X — наименьшее выпуклое мн-во точек, содержащее X. 

## Задача: 

  Если S — множество точек (изначально пустое), $\{p_i\}^n_{i = 1}$ — набор точек, которые последовательно добавляются или удаляются из S (естественно, точка может быть удалена, если она уже принадлежит S).

  Требуется алгоритм, который мог бы поддерживать выпуклую оболочку S в онлайне с возможностью добавления, удаления и локализации точки в выпуклой оболочке.

*Примечание:* В статье описан алгоритм, требующий $O(\log^2{n})$ времени на добавление/удаление/локализацию точки.

## Определения:
Определим левую (правую) выпуклую оболочку множества точек.
> **Левая** *(Правая)* выпуклая оболочка множества точек $P$ — выпуклая оболочка множества точек $P\cup \left\{(+\infty, 0)\right\}$ ($P\cup \left\{(-\infty, 0)\right\}$)

Тогда задачу можно свести к поддержанию отдельно левой и правой выпуклых оболочек. Будем рассматривать только динамическое поддержание левой оболочки. (В правой все так же с точностью до знака)

>Пусть множество точек P разбито горизонтальной прямой на два множества A и C. Рассмотрим выпуклые оболочки A и C. Будем называть **мостом** *(англ. bridge)* отрезок общей касательной к этим оболочкам, заключённый между точками касания. Тогда выпуклая оболочка ко всему множеству P естественным образом получается объединением частей оболочек A и C, и моста B.

Основная идея алгоритма заключается в эффективном (за $O(\log{n})$) отыскании моста, соединяющего две такие оболочки.

<table style="border-style: hidden; border-collapse: collapse;">
<tr style="border-style: hidden; border-collapse: collapse;">
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="Images/left-right_hull.png"><center>Рис 1. Левая и правая выпуклые оболочки</center></div>
    </td>
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="Images/bridge.png"><center>Рис 2. мост между выпуклыми оболочками</center></div>
    </td>
</tr>
</table>

## Структура данных: 

Будем хранить отсортированные лексикографически $(p < q \Leftrightarrow p_y < q_y \lor p_y = q_y \land p_x < q_x)$ точки в листьях сбалансированного бинарного дерева поиска (Например, красно-черного или AVL). 
Во внутренних вершинах будем хранить вспомогательную информацию: 
0. Наименьшую точку в поддереве с корнем в данной вершине; 
0. Мост между выпуклыми оболочками точек левого и правого поддеревьев данной вершины. 

Формально вершина дерева описывается так: 

In [ ]:
class Bridge:
    def __init__(self, p, q):
        self.left = p
        self.right = q

In [ ]:
class Node:
    def __init__(self, point, parent=None, left=None, right=None, bridge=None):
        self.point  = point
        self.parent = parent
        self.left   = left
        self.right  = right
        self.bridge = bridge
    
    def leaf(self):
        return self.left is None and self.right is None

## Объединение выпуклых оболочек 
Пусть, мы хотим объединить оболочки $A$ и $C$, но, для начала наложим на них ограничение - должна существовать горизонтальная прямая, которая их разделяет (как на рисунке 1). Теперь, когда отсортированные точки хранятся в структуре, поддерживающей эффективный поиск, мы можем воспользоватся идеями бинарного поиска для нахождения моста. Для этого необходим критерий спуска, по которому мы будем определять подотрезок, до которого нужно сузить задачу, имея точки $p \in A$ и  $q \in C$, определяющие секущую к выпуклым оболочкам множеств $A$ и  $C$.

Дополнительно определим 4 точки:
>$\textbf p^+$ — следующая за p точка выпуклой оболочки множества A в отсортированном порядке, то есть $p_{y}^{+} > p_{y}$ и <br> ордината $p_{y}$ — наименьшая ,<br>

>$\textbf p^{-}$ — предыдущая p точка выпуклой оболочки множества A в отсортированном порядке, то есть $p_{y}^{-} < p_{y}$ и <br>ордината $p_{y}$ — наибольшая.

>Аналогично определяются $q^+$ и $q^-$.

Так же, определим отношение между вектором и точкой "справа":
> Функция $rot(\overline{ab}, c)$ —  поворот вектора $\overline{ab}$ с точкой $c$.
Тогда,  вектор $\overline{ab}$ находится **"справа"** от точки $c$ если $rot(\overline{ab}, c) > 0$ (т.е точка справа от вектора).

Далее будем обозначать:
> $CH(Q)$ — выпуклая оболочка множества $Q$.

Тогда в зависимости от взаимного расположения вектора $\overrightarrow{qp}$ и точек $p^{+}, p^{-}, q^{+}, q^{-}$ можно определить, какой из 9 случаев рассматривать.
###### Рассмотрим все возможные случаи расположения этих точек:
0. Точки $p^+, p^-, q^+, q^-$ лежат справа от вектора $\overline{qp}$  —  следовательно, отрезок $qp$  —  искомый мост.
0. $p^+, p^-, q^+$  —  справа, а $q^+$  —  слева от $\overline{qp}$. Тогда отбрасываем $CH(A) < p$ и $CH(C) > q$.
0. $p^{+}$, $p^{-}$, $q^{-}$ — справа, $q^{+}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$ и $CH(C) < q$.
0. Как в случае 2, только слева от $\overline{qp}$ находится точка $p^{+}$. Отбрасываем $CH(A) < p$ и $CH(C) > q$.
0. Как в случае 3, только слева от $\overline{qp}$ находится точка $p^{-}$. Отбрасываем $CH(A) > p$ и $CH(C) > q$.
0. $p^{-}$, $q^{+}$ — справа, $p^{+}$, $q^{-}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$ и $CH(C) > q$.
0. $p^{+}$, $q^{+}$ — справа, $p^{-}$, $q^{-}$ — слева от $\overline{qp}$. Отбрасываем $CH(C) > q$.
0. $p^{-}$, $q^{-}$ — справа, $p^{+}$, $q^{+}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$.
0. $p^{+}$, $q^{-}$ — справа, $p^{-}$, $q^{+}$ — слева от $\overline{qp}$. Этот случай дробится на два подслучая.

//TODO: Добавить отрисовку случаев 1-9

Рассмотрим подслучаи случая 9. Пусть $m$ — прямая, разбивающая $P$ на $A$ и $C$. Пусть также $l_p$ и $l_q$ — касательные к выпуклым оболочкам в точках $p$ и $q$ соответственно. Если $s$ — точка пересечения $l_p$ и $l_q$ лежит ниже прямой $m$, то точка пересечения моста и выпуклой оболочки $A$ может лежать в треугольнике, образованном прямыми $l_p$, $l$ и $m$, или выше $p$. Тогда можем удалить часть $C$ ниже $q$ (см. рис. 9.1). Случай, когда $s$ лежит выше $m$, аналогичен (см. рис. 9.2).

//TODO: Добавить отрисовку случаев 9.1-9.2

## Получение выпуклой оболочки

Научимся восстанавливать выпуклую оболочку по дереву, описанному выше. Для этого рассмотрим более общую задачу: для данной вершины дерева $v$ и отрезка $[l, r]$ найти часть выпуклой оболочки, состоящей из точек поддерева с корнем в $v$, ординаты которых лежат в этом отрезке. Это можно сделать следующим обходом: 

In [ ]:
def _get_hull(answer, self, l=float('inf'), r=float('inf')):
    if self.leaf():
        return
    a = self.bridge.left.y
    b = self.bridge.right.y
    if (l < a):
        get_hull(answer, self.left, l, min(a, r))
    if (l <= a and b <= r):
        if (answer.empty()):
            answer = answer.append(self.bridge.left)
        answer = answer.append(self.bridge.right)
    if (b < r):
        get_hull(answer, self.right, max(l, b), r)
def get_hull(self):
    ans = []
    qq = self._get_hull(ans)
    print(qq)
    return qq

setattr(Node, '_get_hull', _get_hull)
setattr(Node, 'get_hull', get_hull)

Левый конец моста добавляется только если ответ пустой, иначе он уже был добавлен как правый конец другого моста. Чтобы получить выпуклую оболочку нужно вызвать *get_hull([ ], root, $-\infty, +\infty$). *

## Добавление точки
Вставим точку в дерево, как в обычное сбалансированное дерево поиска. После этого необходимо пересчитать мосты для некоторых вершин дерева. В вершине необходимо пересчитать мост, либо если в его поддерево была вставлена вершина, либо если при балансировке в повороте эта вершина была задействована (таких вершин $O(\log{n})$. Мосты в таких вершинах будем пересчитывать при подъеме по дереву после вставки вершины. Глубина дерева $O(\log{n})$, значит в процессе подъема будут пересчитаны мосты у $O(\log{n})$ вершин. Каждый пересчет моста требует $O(\log{n})$ времени, откуда итоговая асимптотика вставки: $O(\log^2{n})$. 

## Удаление точки

Удаление точки производится по аналогии с вставкой: сначала мы удалим точку из сбалансированного дерева поиска, а затем пересчитаем мосты, хранящиеся в вершинах, которые затронула балансировка. Очевидно, что удаление, как и вставка, работает за $O(\log^2{n})$. 

## Локализация точки

Cведем запрос к более общему запросу: для данной вершины дерева $v$ и отрезка $[l, r]$ проверить, принадлежит ли точка p части выпуклой оболочки, состоящей из точек поддерева с корнем в $v$, ординаты которых лежат в этом отрезке. Тогда принадлежность точки проверяется вызовом *in_hull(p, root, $-\infty, +\infty$)* 